In [1]:
import os
from dotenv import load_dotenv

load_dotenv('api.env')

HUGGINGFACE_API_KEY = os.environ['HUGGINGFACE_API_KEY']
discord_token = os.environ['discord_token']

In [2]:
from indox import IndoxRetrievalAugmentation
indox = IndoxRetrievalAugmentation()

INFO: IndoxRetrievalAugmentation initialized

            ██  ███    ██  ██████   ██████  ██       ██
            ██  ████   ██  ██   ██ ██    ██   ██  ██
            ██  ██ ██  ██  ██   ██ ██    ██     ██
            ██  ██  ██ ██  ██   ██ ██    ██   ██   ██
            ██  ██  █████  ██████   ██████  ██       ██
            


In [3]:
from indox.llms import HuggingFaceModel
from indox.embeddings import HuggingFaceEmbedding
mistral_qa = HuggingFaceModel(api_key=HUGGINGFACE_API_KEY,model="mistralai/Mistral-7B-Instruct-v0.2")
embed = HuggingFaceEmbedding(api_key=HUGGINGFACE_API_KEY,model="multi-qa-mpnet-base-cos-v1")

INFO: Initializing HuggingFaceModel with model: mistralai/Mistral-7B-Instruct-v0.2
INFO: HuggingFaceModel initialized successfully
INFO: Initialized HuggingFaceEmbedding with model: multi-qa-mpnet-base-cos-v1


In [4]:
from indox.data_connector import DiscordChannelReader
import nest_asyncio

nest_asyncio.apply()
reader = DiscordChannelReader(bot_token=discord_token)

channel_ids = [1275046109722447915]
documents = reader.load_content(channel_ids=channel_ids, num_messages=50)

[2024-08-22 07:19:15] [INFO    ] discord.client: logging in using static token
[2024-08-22 07:19:17] [INFO    ] discord.gateway: Shard ID None has connected to Gateway (Session ID: 516e2f746b2d8921dc4e2e9b6f008a0b).


In [5]:
content = documents


In [7]:
from indox.splitter import semantic_text_splitter
content_chunks = semantic_text_splitter(content,100)

In [8]:
from indox.vector_stores import Chroma
db = Chroma(collection_name="sample",embedding_function=embed)
indox.connect_to_vectorstore(vectorstore_database=db)

INFO: Connection to the vector store database established successfully


In [9]:
indox.store_in_vectorstore(docs= content_chunks)

INFO: Storing documents in the vector store
INFO: Embedding documents
INFO: Starting to fetch embeddings for texts using model: SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: MPNetModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
)
INFO: Document added successfully to the vector store.
INFO: Documents stored successfully
